In [1]:
%config InlineBackend.figure_format = 'svg'
import torch
import sys
import numpy as np

import matplotlib.pyplot as plt
from scipy.io import loadmat

sys.path.append('../')
from models import FNN1d, FNN_train, construct_model, compute_1dFourier_bases
torch.set_printoptions(precision=16)

torch.manual_seed(0)
np.random.seed(0)

# Load data

In [2]:
data_path = "../mytest/data/burgers_data_R10.mat"
data = loadmat(data_path)
data_in = data["a"]
data_out = data["u"]
print("data size = ", data_in.shape[0], " mesh elements = ", data_in.shape[1])


data size =  2048  mesh elements =  8192


In [3]:
torch.manual_seed(0)
np.random.seed(0)

L = 1.0
Ne_ref = data_in.shape[1]
grid = np.linspace(0, L, Ne_ref + 1)[:-1]

downsample_ratio = 2**5
n_train = n_test = 2**10

x_train = torch.from_numpy(
    np.stack(
        (
            data_in[0:n_train, 0::downsample_ratio],
            np.tile(grid[0::downsample_ratio], (n_train, 1)),
        ),
        axis=-1,
    ).astype(np.float32)
)
y_train = torch.from_numpy(
    data_out[0:n_train, 0::downsample_ratio, np.newaxis].astype(np.float32)
)
x_test = torch.from_numpy(
    np.stack(
        (
            data_in[-n_test:, 0::downsample_ratio],
            np.tile(grid[0::downsample_ratio], (n_train, 1)),
        ),
        axis=-1,
    ).astype(np.float32)
)
y_test = torch.from_numpy(
    data_out[-n_test:, 0::downsample_ratio, np.newaxis].astype(np.float32)
)

In [4]:
n_fno_layers = 3
k_max = 33  # 16
d_f = 128
modes = [k_max] * n_fno_layers
layers = [d_f] * (n_fno_layers + 1)
fc_dim = d_f
in_dim = 2
out_dim = 1
act = "gelu"

epochs = 1000
base_lr = 0.001


milestones = [200, 300, 400, 500, 800, 900]
scheduler_gamma = 0.5

scheduler = "MultiStepLR"
weight_decay = 1.0e-4
dim = 1

# weight_decay = 1.0e-4
batch_size = 8

normalization_x = True
normalization_y = True
normalization_dim = []
pad_ratio = 0.05


basis_type = "Galerkin_bases"

if basis_type == "Fast_Fourier_Transform":
    k_max = k_max // 2  # 16
    modes = [k_max] * n_fno_layers

    bases = None
    wbases = None
    model_type = "FNO"

elif basis_type == "Fourier_bases":
    Ne = Ne_ref // downsample_ratio
    grid, fbases, weights = compute_1dFourier_bases(Ne, k_max, L)
    wfbases = fbases * np.tile(weights, (k_max, 1)).T
    bases = [torch.from_numpy(fbases.astype(np.float32))]
    wbases = [torch.from_numpy(wfbases.astype(np.float32))]
    model_type = "GalerkinNO"

elif basis_type == "Galerkin_bases":
    Ne = Ne_ref // downsample_ratio

    pca_data = data_out[0:n_train, 0::downsample_ratio]
    pca_include_input = False
    pca_include_grid = False
    if pca_include_input:
        pca_data = np.vstack((pca_data, data_in[0:n_train, 0::downsample_ratio]))
    if pca_include_grid:
        n_grid = 1
        pca_data = np.vstack(
            (pca_data, np.tile(grid[0::downsample_ratio], (n_grid, 1)))
        )

    U, S, VT = np.linalg.svd(pca_data.T)
    fbases = U[:, 0:k_max] / np.sqrt(L / Ne)
    wfbases = L / Ne * fbases
    bases = [torch.from_numpy(fbases.astype(np.float32))]
    wbases = [torch.from_numpy(wfbases.astype(np.float32))]
    model_type = "GalerkinNO"

else:

    print("Bases construction error")

config = {
    "model": {
        "model": model_type,
        "dim": dim,
        "modes": modes,
        "fc_dim": fc_dim,
        "layers": layers,
        "in_dim": in_dim,
        "out_dim": out_dim,
        "act": act,
        "pad_ratio": pad_ratio,
    },
    "train": {
        "base_lr": base_lr,
        "weight_decay": weight_decay,
        "epochs": epochs,
        "scheduler": scheduler,
        "milestones": milestones,
        "scheduler_gamma": scheduler_gamma,
        "batch_size": batch_size,
        "normalization_x": normalization_x,
        "normalization_y": normalization_y,
        "normalization_dim": normalization_dim,
    },
}


model = construct_model(config, bases, wbases)

In [5]:
print("Start training ", config["model"]["model"])
train_rel_l2_losses, test_rel_l2_losses, test_l2_losses, cost = FNN_train(
    x_train,
    y_train,
    x_test,
    y_test,
    config,
    model,
    save_model_name="../models/save/test.pth",
)

Start training  GalerkinNO
Epoch :  0  Rel. Train L2 Loss :  0.3069101405562833  Rel. Test L2 Loss :  0.16008690232411027  Test L2 Loss :  0.07247034311876632
Epoch :  1  Rel. Train L2 Loss :  0.11749595683068037  Rel. Test L2 Loss :  0.08777787134749815  Test L2 Loss :  0.04001648219127674
Epoch :  2  Rel. Train L2 Loss :  0.07306342566153035  Rel. Test L2 Loss :  0.073866408347385  Test L2 Loss :  0.034203069837531075
Epoch :  3  Rel. Train L2 Loss :  0.05907389981439337  Rel. Test L2 Loss :  0.05598695465596393  Test L2 Loss :  0.024817098659696057
Epoch :  4  Rel. Train L2 Loss :  0.05187512669363059  Rel. Test L2 Loss :  0.058705813687993214  Test L2 Loss :  0.025549183941620868
Epoch :  5  Rel. Train L2 Loss :  0.05691497019142844  Rel. Test L2 Loss :  0.05918113116058521  Test L2 Loss :  0.027704367646947503
Epoch :  6  Rel. Train L2 Loss :  0.05201591248624027  Rel. Test L2 Loss :  0.047992852225434035  Test L2 Loss :  0.021752481647126842


KeyboardInterrupt: 